In [ ]:
"""Processing Corporate Actions using LUSID transitions

Demonstration of booking corporate actions using LUSID's transitions

Attributes
----------
cocoon - seed_data
holdings
transactions
transaction configuration
corporate actions
"""

## Processing Corporate Actions as native LUSID transitions

In this notebook, we will show how you can book corporate action transactions into LUSID using LUSID's native corporate actions functionality. For the purposes of this demo, we will show:

* A dividend payment in cash
* A dividend reinvestment in stock

### Setup LUSID

In [1]:
import os
from datetime import datetime
import pytz

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import format_transactions_response

# Import data wrangling packages
import pandas as pd
import numpy as np
import json

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

### Load a sample portfolio

In this section we seed a new sample portfolio with 10 FTSE100 stocks and GBP cash. This portfolio will be used to demonstrate the corporate actions.

In [2]:
# Create a new scope

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

(For more information on scopes, please see the [scopes](https://support.finbourne.com/what-is-a-scope-in-lusid-and-how-is-it-used) documenation on the support page)

In [3]:
# Load a file of equity transactions

transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [4]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

print(
    f"Portfolio {portfolio_code} has been created with {len(transactions_df)} transactions."
)

Portfolio EQUITY_UK has been created with 21 transactions.


### Build a Transaction Portfolios API

We build a Transactions Portfolios API object so we can interact with the Transaction Portfolio methods. See the [API documentation](https://www.lusid.com/docs/api/#tag/Transaction-Portfolios) for a list of methods. 

In [5]:
# Create a transaction portfolios API

transactions_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [6]:
# View the holdings for today

response = transactions_portfolios_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(response, rename_properties=True)

instrument_uid sub_holding_keys Name(default-Properties)  \
0   LUID_ATFGUBHS               {}                    Aviva   
1   LUID_7XM08GZF               {}                      BHP   
2   LUID_STGB38I6               {}                 Barclays   
3   LUID_PVOJGULG               {}                       BP   
4   LUID_Z57YKL4W               {}                     HSBC   
5   LUID_RNHEK2PL               {}                Morrisons   
6   LUID_99M6G8U7               {}                    Tesco   
7   LUID_6JES517Q               {}                Rightmove   
8   LUID_7KLNIUU7               {}                 vodafone   
9   LUID_NE84MHW9               {}       Anglo American plc   
10        CCY_GBP               {}                  CCY_GBP   

   SourcePortfolioId(default-Properties) holding_type      units  \
0            3861-84ce-5757-d0/EQUITY_UK            P   132000.0   
1            3861-84ce-5757-d0/EQUITY_UK            P   120000.0   
2            3861-84ce-5757-d0/EQUITY_UK            P   300000.0   
3            3861-84ce-5757-d0/EQUITY_UK            P   200000.0   
4            3861-84ce-5757-d0/EQUITY_UK            P    40000.0   
5            3861-84ce-5757-d0/EQUITY_UK            P   360000.0   
6            3861-84ce-5757-d0/EQUITY_UK            P   200000.0   
7            3861-84ce-5757-d0/EQUITY_UK            P   160000.0   
8            3861-84ce-5757-d0/EQUITY_UK            P   900000.0   
9            3861-84ce-5757-d0/EQUITY_UK            P    70000.0   
10           3861-84ce-5757-d0/EQUITY_UK            B  2960000.0   

    settled_units  cost.amount cost.currency  cost_portfolio_ccy.amount  \
0        132000.0     660000.0           GBP                        0.0   
1        120000.0    2160000.0           GBP                        0.0   
2        300000.0     600000.0           GBP                        0.0   
3        200000.0    1000000.0           GBP                        0.0   
4         40000.0     240000.0           GBP                        0.0   
5        360000.0     720000.0           GBP                        0.0   
6        200000.0     400000.0           GBP                        0.0   
7        160000.0     960000.0           GBP                        0.0   
8        900000.0     900000.0           GBP                        0.0   
9         70000.0    1400000.0           GBP                        0.0   
10      2960000.0    2960000.0           GBP                        0.0   

   cost_portfolio_ccy.currency  
0                          GBP  
1                          GBP  
2                          GBP  
3                          GBP  
4                          GBP  
5                          GBP  
6                          GBP  
7                          GBP  
8                          GBP  
9                          GBP  
10                         GBP

### Create corporate actions souce

The corporate actions souce is a container for holding corporate actions. In this section, we:

* Create a new corporate actions source
* Assign corporate actions source to our portfolio

In [7]:
ca_source_code = "ca_demo"

# Create first corporate action source
try:

    source_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code="ca_demo",
        display_name=ca_source_code,
        description="Corporate Actions source for sample notebook",
    )

    source_result = api_factory.build(
        lusid.api.CorporateActionSourcesApi
    ).create_corporate_action_source(create_corporate_action_source_request=source_request)


except:
    pass

In [8]:
# Assign the corporate actions source to our portfolio

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at="2018-01-01T00:00:00+00:00",
    create_portfolio_details=lusid.models.CreatePortfolioDetails(
        corporate_action_source_id=lusid.ResourceId(scope=scope, code=ca_source_code)
    ),
)

print(
    f"Corporate actions source of {ca_source_code} assigned to portfolio {portfolio_code}"
)

Corporate actions source of ca_demo assigned to portfolio EQUITY_UK


### Load transitions from an external source

Transitions determine which Instrument is taking part in a corporate action, and what the effect of the corporate action is on holdings in that Instrument. 

For the transitions in our current example, we have:

* A bonus issue of 13,200 units (1 bonus share per 10 held)
* A cash dividend of £204,000 from BHP (£1.70 GBP per share held)

These transitions are posted with a payment date of 1 March 2020.

In [9]:
# Load transitions fron a CSV file

corporate_action_df = pd.read_csv("data/corp-acts/corpact_transitions.csv")

# Format the datetimes into ISO strings
for col in ["announcement_date", "ex_date", "payment_date", "record_date"]:
    corporate_action_df[col] = corporate_action_df[col].apply(
        lambda x: datetime.strptime(x, "%d/%m/%Y").strftime(format="%Y-%m-%dT00:00:00Z")
    )

corporate_action_df

code action_description description     announcement_date  \
0  5943592342      dividend-cash         BHP  2020-03-31T00:00:00Z   
1  5943592343        bonus-issue       Aviva  2020-03-31T00:00:00Z   

                ex_date           record_date          payment_date client_id  \
0  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1235   
1  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1234   

  input_instrument_name  input_units_factor  input_cost_factor  \
0                   BHP                   1                  1   
1                 Aviva                   1                  1   

  output_instrument_internal  output_units_factor  output_cost_factor  \
0                    CCY_GBP                  1.7                   1   
1                    EQ_1234                  1.1                   1   

   dividend_yield  
0             1.7  
1             0.0

In [10]:
# Add environment specific output LUIDs to the data set

out_instrument_luid = []

for index, item in corporate_action_df.iterrows():
    if item["output_instrument_internal"].startswith("CCY_"):
        out_instrument_luid.append("nan")
    else:
        out_instrument_luid.append(
            api_factory.build(lusid.api.InstrumentsApi)
            .get_instrument(
                identifier_type="ClientInternal",
                identifier=item["output_instrument_internal"],
            )
            .lusid_instrument_id
        )

corporate_action_df["output_instrument_luid"] = out_instrument_luid

In [11]:
# Add environment specific input LUIDs to the data set
# Change the identifier_type as required - this can be any unique identifier which LUSID can resolve to a LUID

corporate_action_df["input_instrument_luid"] = corporate_action_df["client_id"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi)
    .get_instrument(identifier_type="ClientInternal", identifier=x)
    .lusid_instrument_id
)

### Transalate the DataFrame into a LUSID Transition

Here is the transition which needs to be translated into a LUSID `Transition`

In [12]:
corporate_action_df

code action_description description     announcement_date  \
0  5943592342      dividend-cash         BHP  2020-03-31T00:00:00Z   
1  5943592343        bonus-issue       Aviva  2020-03-31T00:00:00Z   

                ex_date           record_date          payment_date client_id  \
0  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1235   
1  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1234   

  input_instrument_name  input_units_factor  input_cost_factor  \
0                   BHP                   1                  1   
1                 Aviva                   1                  1   

  output_instrument_internal  output_units_factor  output_cost_factor  \
0                    CCY_GBP                  1.7                   1   
1                    EQ_1234                  1.1                   1   

   dividend_yield output_instrument_luid input_instrument_luid  
0             1.7                    nan         LUID_7XM08GZF  
1             0.0          LUID_ATFGUBHS         LUID_ATFGUBHS

In [13]:
# This code produces a dictionary of transitions where:
# The dict key is the corporate action code and the dict value is a list of
# input and output CorporateActionTransitionComponentRequest objects


transitions = {}
LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId"

for index, ca in corporate_action_df.iterrows():

    # create transition components
    cat_in = models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={
            LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
        },
        units_factor=ca["input_units_factor"],
        cost_factor=ca["input_cost_factor"],
    )

    # Determine if the output is Cash or Stock, and create the appropriate transition component
    if str(ca["output_instrument_luid"]) == "nan":
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Currency": ca["output_instrument_internal"][4:]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
    else:
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )

    key = ca["code"]
    transitions.setdefault(key, [])
    transitions[key].append(ca["action_description"])
    transitions[key].append(cat_in)
    transitions[key].append(cat_out)

Use the transition to create an [UpsertCorporateActionRequest](https://www.lusid.com/docs/api/#operation/BatchUpsertCorporateActions)

In [14]:
# Iterate through the transitions, turning them into LUSID Corporate Action Requests
for key, values in transitions.items():
    transition_code = key
    transition_type = values[0]
    transition_in = values[1]
    transitions_out = values[2:]

    for x in transitions_out:
        if isinstance(x, str):
            transitions_out.remove(x)

    temp_transition = models.CorporateActionTransition(
        input_transition=transition_in, output_transitions=transitions_out
    )

    # Extract the data for the corporate action from the LUID corporate actions dataframe
    data = corporate_action_df[corporate_action_df["code"] == transition_code]

    # Iterate through each row of data and create the Corporate Action Request for the appropriate type
    # of action.
    for row, item in data.iterrows():
        if transition_type == "dividend-cash":
            div_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        if transition_type == "bonus-issue":
            split_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )

### Upsert the corporate actions into LUSID's movements engine

In [15]:
result = api_factory.build(
    lusid.api.CorporateActionSourcesApi
).batch_upsert_corporate_actions(
    scope=scope, code=ca_source_code, upsert_corporate_action_request=[div_ca, split_ca]
)

### Holdings in Aviva and GBP cash on 31 January 2020 (before the Corporate Action)

* Portfolio has 132,000 units of Aviva
* Portfolio has £2,960,000 GBP cash

In [16]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-01-31",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]
holdings_df[holdings_df.columns[:9:2].to_list()]

instrument_uid Name(default-Properties) holding_type  settled_units  \
0   LUID_ATFGUBHS                    Aviva            P       132000.0   
1   LUID_7XM08GZF                      BHP            P       120000.0   
2   LUID_STGB38I6                 Barclays            P       300000.0   
3   LUID_PVOJGULG                       BP            P       200000.0   
4   LUID_Z57YKL4W                     HSBC            P        40000.0   
5   LUID_RNHEK2PL                Morrisons            P       360000.0   
6   LUID_99M6G8U7                    Tesco            P       200000.0   
7   LUID_6JES517Q                Rightmove            P       160000.0   
8   LUID_7KLNIUU7                 vodafone            P       900000.0   
9   LUID_NE84MHW9       Anglo American plc            P        70000.0   
10        CCY_GBP                  CCY_GBP            B      2960000.0   

   cost.currency  
0            GBP  
1            GBP  
2            GBP  
3            GBP  
4            GBP  
5            GBP  
6            GBP  
7            GBP  
8            GBP  
9            GBP  
10           GBP

### Holdings in Aviva and GBP cash on 30 April 2020 (after the Corporate Action)

* Portfolio has 145,200 units of Aviva (+13,200 units)
* Portfolio has £3,164,000 GBP cash (+ £204,000 cash)

In [17]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-04-30",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]
holdings_df[holdings_df.columns[:9:2].to_list()]

instrument_uid Name(default-Properties) holding_type  settled_units  \
0   LUID_ATFGUBHS                    Aviva            P       145200.0   
1   LUID_7XM08GZF                      BHP            P       120000.0   
2   LUID_STGB38I6                 Barclays            P       300000.0   
3   LUID_PVOJGULG                       BP            P       200000.0   
4   LUID_Z57YKL4W                     HSBC            P        40000.0   
5   LUID_RNHEK2PL                Morrisons            P       360000.0   
6   LUID_99M6G8U7                    Tesco            P       200000.0   
7   LUID_6JES517Q                Rightmove            P       160000.0   
8   LUID_7KLNIUU7                 vodafone            P       900000.0   
9   LUID_NE84MHW9       Anglo American plc            P        70000.0   
10        CCY_GBP                  CCY_GBP            B      3164000.0   

   cost.currency  
0            GBP  
1            GBP  
2            GBP  
3            GBP  
4            GBP  
5            GBP  
6            GBP  
7            GBP  
8            GBP  
9            GBP  
10           GBP